In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,Dense,MaxPooling2D,Flatten,BatchNormalization,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import preprocess_input
import matplotlib.pyplot as plt
import cv2 as cv
from joblib import load
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,classification_report
from keras.utils import to_categorical

import os

In [2]:
train_path_str = 'E:/Lung_Cancer_CNN/Data/Data/Dataset/Train'
val_path_str = 'E:/Lung_Cancer_CNN/Data/Data/Dataset/Valid'
test_path_str = 'E:/Lung_Cancer_CNN/Data/Data/Dataset/Test'

In [3]:
test_classes = os.listdir(test_path_str)

In [4]:
input_shape = (224,224,3)
num_classes = 4

trainGenertor = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    rotation_range = 10,
    width_shift_range = 0.3,
    height_shift_range = 0.3,
    shear_range = 0.2,
    zoom_range = 0.1,
    horizontal_flip = True,
    vertical_flip = True,
    dtype = 'float32'
)
valGenertor = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    dtype = 'float32'
)

testGenertor = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    dtype = 'float32'
)

In [5]:
train_data = trainGenertor.flow_from_directory(
    train_path_str,
    target_size = (224,224),
    batch_size = 16,
    class_mode = 'categorical'
)

val_data = valGenertor.flow_from_directory(
    val_path_str,
    target_size = (224,224),
    batch_size = 16,
    class_mode = 'categorical'
)

test_data = testGenertor.flow_from_directory(
    test_path_str,
    target_size = (224,224),
    batch_size = 16,
    class_mode = 'categorical',
    shuffle = False
)
# plt.imshow(train_data)
# for image in len(tra)
# plotter(i)

Found 613 images belonging to 4 classes.
Found 72 images belonging to 4 classes.
Found 315 images belonging to 4 classes.


In [6]:
VGG16_model = VGG16(
    include_top = False,
    weights="imagenet",
    input_shape = input_shape
)
for layer in VGG16_model.layers :
  layer.trainable = False

In [7]:
from keras.src.metrics.confusion_metrics import activations
model = Sequential()
model.add(VGG16_model)
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(1024,activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(512,activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(256,activation = 'relu'))
model.add(Dense(128,activation = 'relu'))
model.add(Dense(num_classes,activation = 'softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 batch_normalization (Batch  (None, 7, 7, 512)         2048      
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 3, 3, 512)         0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 4608)              0         
                                                                 
 dense (Dense)               (None, 1024)              4719616   
                                                                 
 dropout (Dropout)           (None, 1024)              0

In [8]:
model.compile(
    optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy']
)
callbacks = [
    
    tf.keras.callbacks.ModelCheckpoint('model.joblib',save_best_only=True),
    tf.keras.callbacks.TensorBoard(log_dir='logs')
]

In [ ]:
results = model.fit(train_data,validation_data=val_data,epochs=100,verbose = 1,callbacks=[callbacks])

Epoch 1/100
39/39 [==============================] - ETA: 0s - loss: 1.5268 - accuracy: 0.3948INFO:tensorflow:Assets written to: model.joblib\assets


INFO:tensorflow:Assets written to: model.joblib\assets


39/39 [==============================] - 77s 2s/step - loss: 1.5268 - accuracy: 0.3948 - val_loss: 1.7227 - val_accuracy: 0.4028
Epoch 2/100
39/39 [==============================] - 66s 2s/step - loss: 1.2566 - accuracy: 0.4796 - val_loss: 1.8345 - val_accuracy: 0.3194
Epoch 3/100
39/39 [==============================] - ETA: 0s - loss: 1.0428 - accuracy: 0.5334INFO:tensorflow:Assets written to: model.joblib\assets


INFO:tensorflow:Assets written to: model.joblib\assets


39/39 [==============================] - 70s 2s/step - loss: 1.0428 - accuracy: 0.5334 - val_loss: 0.9542 - val_accuracy: 0.5694
Epoch 4/100
39/39 [==============================] - 67s 2s/step - loss: 0.9893 - accuracy: 0.5481 - val_loss: 1.0118 - val_accuracy: 0.4861
Epoch 5/100
39/39 [==============================] - ETA: 0s - loss: 0.9753 - accuracy: 0.5612INFO:tensorflow:Assets written to: model.joblib\assets


...

39/39 [==============================] - 67s 2s/step - loss: 0.3763 - accuracy: 0.8418 - val_loss: 0.6540 - val_accuracy: 0.8056
Epoch 30/100
39/39 [==============================] - ETA: 0s - loss: 0.4038 - accuracy: 0.8467INFO:tensorflow:Assets written to: model.joblib\assets


INFO:tensorflow:Assets written to: model.joblib\assets


39/39 [==============================] - 67s 2s/step - loss: 0.4038 - accuracy: 0.8467 - val_loss: 0.5640 - val_accuracy: 0.8194
Epoch 31/100
39/39 [==============================] - 62s 2s/step - loss: 0.4103 - accuracy: 0.8515 - val_loss: 0.7084 - val_accuracy: 0.7222
Epoch 32/100
39/39 [==============================] - ETA: 0s - loss: 0.4253 - accuracy: 0.8336INFO:tensorflow:Assets written to: model.joblib\assets


INFO:tensorflow:Assets written to: model.joblib\assets


39/39 [==============================] - 69s 2s/step - loss: 0.4253 - accuracy: 0.8336 - val_loss: 0.5294 - val_accuracy: 0.8333
Epoch 33/100
39/39 [==============================] - ETA: 0s - loss: 0.4132 - accuracy: 0.8499

In [ ]:
loss, acc = model.evaluate(test_data,verbose = 1)

In [ ]:
import seaborn as sns
predictions_prob = model.predict(test_data)
predictions = np.argmax(predictions_prob , axis = 1)
true_label = test_data.classes
report = classification_report(true_label,predictions)
print(report) 

In [ ]:
conf_mat = confusion_matrix(true_label,predictions)
sns.heatmap(conf_mat ,fmt='g',annot = True , cmap='Blues' , xticklabels=test_classes , yticklabels = test_classes,)
plt.xlabel('Predictions')
plt.ylabel('True label')
plt.title('Confusion Matrix')
plt.xticks(rotation = 45)
plt.show()

In [ ]:

import joblib 

model_directory_path = 'E:/Lung_Cancer_CNN/Data/Data/model.joblib/'
from tensorflow.keras.models import load_model
'

model = load_model(model_directory_path)

